In [10]:
%reload_ext autoreload
%autoreload 2

import torch

torch.manual_seed(42)


In [11]:
from learning.gpt2.model import GPT2, PretrainedName


device = torch.device("cuda")
model, pretrained_model = GPT2.from_pretrained(PretrainedName.GPT2_SMALL, device=device)
pretrained_model = pretrained_model.to(device)


In [12]:
import tiktoken
from transformers.modeling_outputs import CausalLMOutputWithCrossAttentions

tokenizer = tiktoken.get_encoding("gpt2")


def generate(model, s: str, times: int):
    model.eval()
    with torch.no_grad():
        indices = tokenizer.encode(s)
        inputs = torch.tensor([indices], dtype=torch.long, device=device)

        for _ in range(times):
            outputs = model.generate(inputs, 2)
            for i in range(outputs.shape[0]):
                print(tokenizer.decode(outputs[i].tolist()))


def top_logits(model, s: str, k: int):
    print(s)
    model.eval()
    with torch.no_grad():
        indices = tokenizer.encode(s)
        inputs = torch.tensor([indices], dtype=torch.long, device=device)

        outputs = model(inputs)
        if isinstance(outputs, CausalLMOutputWithCrossAttentions):
            outputs = outputs.logits
            assert isinstance(outputs, torch.Tensor)
        else:
            assert isinstance(outputs, torch.Tensor)

        last_output = outputs[:, -1, :]

        probs = torch.softmax(last_output, dim=-1).squeeze(0)
        top_probs, top_indices = torch.topk(probs, k=k)
        for i in range(k):
            print(f"{top_probs[i]:.2f} {tokenizer.decode([int(top_indices[i])])}")

In [14]:
def run_test(model):
    # From the paper: https://arxiv.org/abs/2211.00593
    cases = [
        "When Mary and John went to the store, John gave a drink to",
        "When Vincent and Vanessa went to the park, Vincent gave a leaf to",
        "When Vincent and Vanessa went to the park, Vanessa gave a leaf to",
        "Mary and John went to the store. John gave a drink to",
        "Mary and John went to the store; John gave a drink to",
        "Mary and John went to the store! John gave a drink to",
        "Mary and John went to the store. Mary gave a drink to",
        "Mary and John went to the store; Mary gave a drink to",
        "Mary and John went to the store! Mary gave a drink to",
    ]

    for case in cases:
        top_logits(model, case, 3)


run_test(model)

When Mary and John went to the store, John gave a drink to
0.45  Mary
0.21  them
0.07  the
When Vincent and Vanessa went to the park, Vincent gave a leaf to
0.51  Vanessa
0.12  the
0.06  a
When Vincent and Vanessa went to the park, Vanessa gave a leaf to
0.54  Vincent
0.09  the
0.03  her
Mary and John went to the store. John gave a drink to
0.31  them
0.15  John
0.13  the
Mary and John went to the store; John gave a drink to
0.33  them
0.15  John
0.14  the
Mary and John went to the store! John gave a drink to
0.27  them
0.13  the
0.10  John
Mary and John went to the store. Mary gave a drink to
0.46  John
0.14  them
0.12  the
Mary and John went to the store; Mary gave a drink to
0.29  John
0.21  them
0.14  the
Mary and John went to the store! Mary gave a drink to
0.40  John
0.14  the
0.11  them
